In [1]:
import ray
ray.init(ignore_reinit_error=True)

{'node_ip_address': '10.160.0.3',
 'raylet_ip_address': '10.160.0.3',
 'redis_address': '10.160.0.3:6379',
 'object_store_address': '/tmp/ray/session_2021-11-14_21-55-27_449565_105374/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-11-14_21-55-27_449565_105374/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-11-14_21-55-27_449565_105374',
 'metrics_export_port': 56385,
 'node_id': 'fc6def2b76f66959965a34b10df5004779910179984c64f10920f4a6'}

In [6]:
@ray.remote
def run_exec():
    import sys
    sys.path.append('../')
    from fl_strategies.fed_avg import FedAvg
    import yaml
    from experiments import dlrm 
    from fl_strategies import fed_avg
    from fedrec.utilities.logger import NoOpLogger
    from fedrec.python_executors.aggregator import AggregatorConfig

    from fedrec.multiprocessing.process_manager import RayProcessManager
    from executor import JobExecutor
    with open("../configs/dlrm_fl.yml", 'r') as cfg:
        config = yaml.load(cfg, Loader=yaml.FullLoader)

    ag_config = {
            # Seed for RNG used in shuffling the training data.
        "data_seed" : 100,
        # Seed for RNG used in initializing the model.
        "init_seed" : 100,
        # Seed for RNG used in computing the model's training loss.
        # Only relevant with internal randomness in the model, e.g. with dropout.
        "model_seed" : 100
    }
    aggregator_cfg = AggregatorConfig(**ag_config)
    JobExecutor = ray.remote(JobExecutor)
    je_Aggregator = JobExecutor.remote(FedAvg,
                                    config,
                                    aggregator_cfg,
                                    NoOpLogger())
    je_Aggregator.run.remote()

run_exec.remote()

ObjectRef(69a6825d641b4613ffffffffffffffffffffffff0100000001000000)

(pid=105961) WARNING <class 'fedrec.communications.kafka_interfaces.Kafka'>: superfluous {'cosnumer_group_id': 'My-group-id'}
(pid=105961) 2021-11-14 21:58:19,911	ERROR worker.py:425 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::JobExecutor.__init__() (pid=105961, ip=10.160.0.3)
(pid=105961)   File "/home/varun/RecoEdge/notebooks/../executor.py", line 29, in __init__
(pid=105961)     self.jobber = Jobber(
(pid=105961)   File "/home/varun/RecoEdge/notebooks/../fedrec/multiprocessing/jobber.py", line 28, in __init__
(pid=105961)     self.comm_manager = registry.construct(
(pid=105961)   File "/home/varun/RecoEdge/notebooks/../fedrec/utilities/registry.py", line 31, in construct
(pid=105961)     return instantiate(
(pid=105961)   File "/home/varun/RecoEdge/notebooks/../fedrec/utilities/registry.py", line 59, in instantiate
(pid=105961)     return callable(**merged)
(pid=105961)   File "/home/varun/RecoEdge/notebooks/../fedrec/co

In [5]:
ray.shutdown()

In [ ]:
rpm = RayProcessManager()
rpm.distribute(JobExecutor, FedAvg.__name__, 1 ,
                            FedAvg, config["model"], aggregator_cfg, NoOpLogger())

rpm.start(FedAvg.__name__, "run")

In [ ]:
ray.shutdown()